In [1]:
import os
import pandas as pd

In [2]:
main_directory = "C:/Users/PC/Documents/ABC_2025/output_csvs_test"  

In [3]:
def segment_data_by_time(df, segment_duration=3):
    segments = []
    start_time = df['Timestamp'].iloc[0]
    end_time = df['Timestamp'].iloc[-1]

    current_time = start_time

    while current_time + pd.Timedelta(seconds=segment_duration) <= end_time:
        segment = df[(df['Timestamp'] >= current_time) & (df['Timestamp'] < current_time + pd.Timedelta(seconds=segment_duration))]
        segments.append(segment)
        current_time += pd.Timedelta(seconds=segment_duration)

    return segments

In [4]:
def load_data(directory):
    all_segments = []
    for file_name in os.listdir(directory):
        if file_name.endswith('.csv'):
            try:
                file_path = os.path.join(directory, file_name)
                df = pd.read_csv(file_path)
                df.columns = ['MotionType', 'Timestamp', 'X', 'Y', 'Z'] 

                df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='ISO8601', errors='coerce')

                df = df.drop(columns=['MotionType'])

                segments = segment_data_by_time(df, segment_duration=3)
                all_segments.append((file_name, segments))  

            except Exception as e:
                print(f"Error reading {file_name}: {e}")

    return all_segments

In [7]:
output_directory = os.path.join(main_directory, 'segment_test')
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

In [11]:
all_segments = load_data(main_directory)

In [12]:
for file_name, segments in all_segments:  
    for i, segment in enumerate(segments):
        base_name = os.path.splitext(file_name)[0]  
        segment_file_name = f"{base_name}_{i + 1}.csv" 
        segment_file_path = os.path.join(output_directory, segment_file_name)
        segment.to_csv(segment_file_path, index=False)
        
print("Data segmentation completed and saved.")

Data segmentation completed and saved.
